In [1]:

import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, RepeatedKFold, LeaveOneOut
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.linear_model import HuberRegressor, LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_predict
import numpy as np, seaborn as sns, matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import os
import sys

from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler

# Add project root to path (for Jupyter notebooks)
# Get the current directory and navigate to project root
current_dir = os.getcwd()
project_root = os.path.dirname(current_dir)
sys.path.insert(0, project_root)

from src.config import Config
import src.eda as eda
import src.present_value as present_value

%load_ext autoreload
%autoreload 2
%reload_ext autoreload


c:\Users\forer\anaconda3\envs\practicum\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [3]:
## FROM DATABASE
pv = present_value.PresentValue()
anual_increment = pv.fetch_incremento_from_database()

fase = "III"
preproccesing = eda.EDA()
df_raw = preproccesing.assemble_projects_from_database(fase)
df_vp = preproccesing.create_dataset(pv.present_value_costs, fase=fase)

c:\Code\Road-Project-Prediction\src\eda.py:292: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  w = (df[cols] / totals).fillna(0)


In [2]:
def remove_outliers(df, target: str, method='ensemble', contamination=0.1, voting_threshold=0.5) -> pd.DataFrame:
    """
    Advanced outlier detection using multiple methods.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    target : str
        Target column name
    method : str
        'ensemble' (default): Combines multiple methods with voting
        'isolation_forest': Uses Isolation Forest only
        'lof': Uses Local Outlier Factor only
        'robust_statistical': Uses Modified Z-score with MAD
        'all_strict': All methods must agree (strictest)
    contamination : float
        Expected proportion of outliers (0.05-0.2 typical)
    voting_threshold : float
        For ensemble method, fraction of methods that must flag as outlier (0.5 = majority)
    
    Returns:
    --------
    pd.DataFrame
        Cleaned dataframe without outliers
    """
    
    # Remove zero values first (domain-specific)
    df_nonzero = df[df[target] != 0].copy()
    
    if len(df_nonzero) < 10:
        print(f"Warning: Only {len(df_nonzero)} samples. Skipping outlier detection.")
        return df_nonzero
    
    # Prepare features: numerical columns + target
    numerical_cols = df_nonzero.select_dtypes(include=[np.number]).columns.tolist()
    
    # Scale features for better outlier detection
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df_nonzero[numerical_cols])
    
    # Initialize outlier flags
    outlier_flags = {}
    
    # Method 1: Isolation Forest (excellent for high-dimensional data)
    if method in ['ensemble', 'isolation_forest', 'all_strict']:
        iso_forest = IsolationForest(
            contamination=contamination,
            random_state=42,
            n_estimators=200,
            max_samples='auto',
            bootstrap=True
        )
        iso_predictions = iso_forest.fit_predict(X_scaled)
        outlier_flags['isolation_forest'] = (iso_predictions == -1)
        
    # Method 2: Local Outlier Factor (density-based, good for local anomalies)
    if method in ['ensemble', 'lof', 'all_strict']:
        n_neighbors = min(20, len(df_nonzero) - 1)
        lof = LocalOutlierFactor(
            n_neighbors=n_neighbors,
            contamination=contamination
        )
        lof_predictions = lof.fit_predict(X_scaled)
        outlier_flags['lof'] = (lof_predictions == -1)
    
    # Method 3: Modified Z-score with MAD (robust to outliers themselves)
    if method in ['ensemble', 'robust_statistical', 'all_strict']:
        target_values = df_nonzero[target].values
        median = np.median(target_values)
        mad = np.median(np.abs(target_values - median))
        
        # Modified Z-score (more robust than standard Z-score)
        if mad != 0:
            modified_z_scores = 0.6745 * (target_values - median) / mad
            # Threshold of 3.5 is standard for modified Z-score
            outlier_flags['robust_statistical'] = np.abs(modified_z_scores) > 3.5
        else:
            outlier_flags['robust_statistical'] = np.zeros(len(df_nonzero), dtype=bool)
    
    # Method 4: Multivariate Z-score on target (additional check)
    if method in ['ensemble', 'all_strict']:
        target_scaled = scaler.fit_transform(df_nonzero[[target]])
        outlier_flags['z_score'] = np.abs(target_scaled.flatten()) > 3
    
    # Combine methods based on selected strategy
    if method == 'ensemble':
        # Voting: flag as outlier if voting_threshold fraction of methods agree
        outlier_matrix = np.column_stack(list(outlier_flags.values()))
        votes = outlier_matrix.sum(axis=1)
        is_outlier = votes >= (len(outlier_flags) * voting_threshold)
        
    elif method == 'all_strict':
        # All methods must agree (most conservative)
        outlier_matrix = np.column_stack(list(outlier_flags.values()))
        is_outlier = outlier_matrix.all(axis=1)
        
    elif method in ['isolation_forest', 'lof', 'robust_statistical']:
        # Single method
        is_outlier = outlier_flags[method]
    
    else:
        raise ValueError(f"Unknown method: {method}")
    
    # Filter out outliers
    df_clean = df_nonzero[~is_outlier].copy()
    
    # Report
    n_outliers = is_outlier.sum()
    pct_removed = (n_outliers / len(df_nonzero)) * 100
    print(f"  → Removed {n_outliers}/{len(df_nonzero)} outliers ({pct_removed:.1f}%) using {method}")
    
    if method == 'ensemble' and len(outlier_flags) > 1:
        for flag_name, flags in outlier_flags.items():
            print(f"     • {flag_name}: {flags.sum()} outliers")
    
    return df_clean

In [6]:
df_vp[df_vp['4 - SUELOS']!=0]

,NOMBRE DEL PROYECTO,CÓDIGO,LONGITUD KM,PUENTES VEHICULARES UND,PUENTES VEHICULARES M2,PUENTES PEATONALES UND,PUENTES PEATONALES M2,TUNELES UND,TUNELES KM,ALCANCE,...,7 - SOCAVACIÓN,8 - ESTRUCTURAS,9 - TÚNELES,10 - URBANISMO Y PAISAJISMO,11 - PREDIAL,12 - IMPACTO AMBIENTAL,13 - CANTIDADES,14 - EVALUACIÓN SOCIOECONÓMICA,15 - OTROS - MANEJO DE REDES,16 - DIRECCIÓN Y COORDINACIÓN
0,TAMBILLO - COLIBRÍ,0001,17.400,11,4856,14,1365,0,0.0000,Segunda calzada,...,5.275975e+07,6.838215e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.377238e+07,0.000000e+00,5.027965e+07,1.714043e+08
1,DCO,0004,13.690,0,0,0,0,0,0.0000,Nuevo,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.634289e+08
9,CATAMBUCO - PASTO UF5.2,0266702,5.000,0,0,1,58,0,0.0000,Mejoramiento,...,6.289853e+06,6.421774e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.901125e+07,6.672842e+06
10,BUGA - BUENAVENTURA EL PORVENIR UF5,0300604,2.500,0,0,0,0,0,0.0000,Segunda calzada,...,9.159642e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.250960e+07
11,BUGA - BUENAVENTURA UF4,0300605,17.000,13,16654,0,0,0,0.0000,Segunda calzada,...,4.467656e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.775515e+07
20,TERMINACIÓN TÚNEL DE LA LÍNEA,0347801,8.350,10,24258,0,0,5,0.9200,Rehabilitación,...,6.284023e+07,4.415680e+08,6.644646e+07,0.000000e+00,1.663940e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.006346e+08
21,TERMINACIÓN TÚNEL DE LA LÍNEA,0347801,8.600,0,0,0,0,1,8.6000,Rehabilitación,...,6.472167e+07,4.547886e+08,1.632824e+08,0.000000e+00,1.713759e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.066416e+08
22,TERMINACIÓN TÚNEL DE LA LÍNEA,0347801,8.500,16,26811,0,0,15,2.2747,Rehabilitación,...,6.396909e+07,4.495003e+08,1.906902e+08,0.000000e+00,1.693831e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.042388e+08
23,PURIFICACIÓN - LOZANÍA,0427901,5.000,0,0,0,0,0,0.0000,Mejoramiento,...,1.321083e+07,9.507830e+06,0.000000e+00,0.000000e+00,0.000000e+00,1.653386e+06,6.853860e+06,0.000000e+00,0.000000e+00,5.695082e+06
24,TRANSVERSAL BOYACÁ,0552903,10.060,6,3837,0,0,0,0.0000,Mejoramiento,...,3.596050e+07,2.169088e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.513204e+06,0.000000e+00,0.000000e+00,2.202194e+07
